In [3]:
from dotenv import load_dotenv
import os
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, ToolMessage
from operator import add as add_messages



from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb
import requests
from langchain_core.tools import tool

load_dotenv()

True

In [1]:
from langchain_community.document_loaders import PyPDFLoader
pdf_path ='pdf/Fundamentals of Building Autonomous LLM.pdf'
pages = PyPDFLoader(pdf_path).load()
len(pages)

c:\Users\mawiy\OneDrive\Desktop\Legal-Agent\myLegal\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


38

In [4]:
text_splitter= RecursiveCharacterTextSplitter(chunk_size =1000, chunk_overlap=200,separators=[
        "\n\n",
        "\n",
        " ",
        ""
        ])

pages_split=text_splitter.split_documents(pages)

In [5]:
persist_directory = r"C:\Users\mawiy\OneDrive\Desktop\Legal-Agent\learning Langraph\pdf"

collection_name = "cases"

# If our collection does not exist in the directory, we create using the os command
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

In [6]:
print(pages_split)

[Document(metadata={'producer': 'pikepdf 8.15.1', 'creator': 'arXiv GenPDF (tex2pdf:e76afa9)', 'creationdate': '', 'author': 'Victor de Lamo Castrillo; Habtom Kahsay Gidey; Alexander Lenz; Alois Knoll', 'doi': 'https://doi.org/10.48550/arXiv.2510.09244', 'license': 'http://creativecommons.org/licenses/by-nc-nd/4.0/', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.28 (TeX Live 2025) kpathsea version 6.4.1', 'title': 'Fundamentals of Building Autonomous LLM Agents', 'trapped': '/False', 'arxivid': 'https://arxiv.org/abs/2510.09244v1', 'source': 'pdf/Fundamentals of Building Autonomous LLM.pdf', 'total_pages': 38, 'page': 0, 'page_label': '1'}, page_content='Fundamentals of Building Autonomous LLM\nAgents ⋆\nVictor de Lamo Castrillo1 , Habtom Kahsay Gidey2 , Alexander Lenz2 , and\nAlois Knoll2\n1 Universitat Politècnica de Catalunya, Barcelona, Spain\nvictor.de.lamo@estudiantat.upc.edu\n2 Technische Universität München, München, Germany\n{habtom.gidey, alex.lenz, knoll}@

In [7]:
from langchain_community.embeddings import JinaEmbeddings
jina = JinaEmbeddings(
                api_key=os.getenv('JINA_API_KEY'),
                model_name="jina-embeddings-v3",
                
            )
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents = pages_split,
    persist_directory= persist_directory,
    collection_name=collection_name,
    embedding=jina
)

def retriever(str) :
    ret= vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )
    return ret.invoke(str)




In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm= ChatGoogleGenerativeAI(model='gemini-2.0-flash')


In [9]:
from langchain_classic.retrievers import EnsembleRetriever
from langchain_classic.retrievers import BM25Retriever

bm25_retriver = BM25Retriever.from_documents(pages_split)




In [15]:
ret= vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    )

In [16]:
ensemble_ret=EnsembleRetriever(retrievers=[ret,bm25_retriver],
                               weights=[0.5,0.5]
                               )

In [21]:
query="What does the acronym DPPM stand for in the context of reasoning?"
results = ensemble_ret.invoke(query)

In [ ]:
for i,doc in enumerate(results):
    print("Doc :",i+1)
    print('content :' ,doc.page_content)
    print("metadata :",doc.metadata)
    

Doc : 1
content : DPPM (Decompose, Plan in Parallel, and Merge). It addresses the limitations of
existing planning methods, such as:
1. Handling heavy constraints
2. Carrying errors from the planning of previous steps
3. Forgetting the main goal
4. Cohesion between subtasks
DPPM tackles these problems with the following methods: First, it decomposes
the complex task into subtasks. Second, it generates subplans for each of these
subtasks concurrently using individual LLM agents. This parallel planning allows
each agent to focus only on its assigned subtask, promoting independent work
and avoiding the cascading errors that can occur when subplans are sequentially
dependent. Finally, DPPM merges these independently generated local subplans
into a coherent global plan [36]. Although this method can struggle to adapt
well to unexpected environmental problems, this limitation can be mitigated by
reflecting on the plan after each execution step.
metadata : {'arxivid': 'https://arxiv.org/abs/2

In [26]:
answer=llm.invoke(f'this is the relevent context {results} based on this and your understanding give me appropriate answer best to your knowledge ')
print(answer.content)

Okay, based on the provided context, here's a summary of DPPM (Decompose, Plan in Parallel, and Merge) and its role in LLM agents, along with related concepts:

**DPPM: Decompose, Plan in Parallel, and Merge**

*   **What it is:** DPPM is a reasoning system designed to improve the planning capabilities of LLM agents. It addresses limitations found in other planning methods.

*   **The Problem it Solves:**
    *   Handling complex constraints.
    *   Cascading errors from previous planning steps.
    *   Forgetting the main goal.
    *   Lack of cohesion between subtasks.

*   **How it Works:**
    1.  **Decompose:** The complex task is broken down into smaller, manageable subtasks.
    2.  **Plan in Parallel:** Individual LLM agents generate subplans for each subtask concurrently.  This allows each agent to focus on its specific subtask, preventing cascading errors that can occur with sequential planning.  During this stage, potential issues are considered, and alternative approaches 